# Imports, load data

In [ ]:
import numpy as np
import tensorflow as tf
from distance_determination import get_current_freq
from simul.utilities.data import load_experiment
from simul.vis.signals import vis_signals
from tqdm.auto import tqdm

In [2]:
params, dist, signals_data, dist_probs = load_experiment("default_full")

# VIS

In [ ]:
# fig_amp, fig_angle = vis_signals(signals_data, dist, params, 500, dump=True)

# fig_amp.update_layout(height=800)
# fig_amp.show()

# fig_angle.update_layout(height=800)
# fig_angle.show()

# Prepare the train dataset

In [52]:
signals_data_with_skips = signals_data.copy()
for ts_i, ts in enumerate(tqdm(params.tss, leave=False)):
    curr_freq = get_current_freq(ts_i, params)
    ids = np.ones(signals_data_with_skips.shape[0], dtype=bool)
    ids[round(curr_freq / 2)] = 0
    signals_data_with_skips[ids, ts_i] = np.NaN

assert (~np.isnan(signals_data_with_skips)).sum() == signals_data_with_skips.shape[1]

  0%|          | 0/23336 [00:00<?, ?it/s]

In [53]:
n = signals_data.shape[1]
train_signals_data = signals_data_with_skips[:, 0 : int(n * 0.7)]
val_signals_data = signals_data_with_skips[:, int(n * 0.7) : int(n * 0.9)]
test_signals_data = signals_data_with_skips[:, int(n * 0.9) :]

train_labels = signals_data[:, 0 : int(n * 0.7)]
val_labels = signals_data[:, int(n * 0.7) : int(n * 0.9)]
test_labels = signals_data[:, int(n * 0.9) :]

batch_size = 32
train_signals_data=train_signals_data.T.reshape((bat))
signals_data=signals_data[None, ...]

In [56]:
train_signals_data.T.shape

(16335, 40)

In [ ]:
window_size = 6
train = np.ndarray()

# Create a neural network

In [31]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(
            40,
            return_sequences=False,
            stateful=True,
            recurrent_initializer="glorot_uniform",
        ),
        tf.keras.layers.Dense(40),
    ]
)

2022-01-19 09:07:16.300311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-19 09:07:16.300336: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-19 09:07:16.300352: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (astadnik-xps139310): /proc/driver/nvidia/version does not exist
2022-01-19 09:07:16.300571: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["accuracy"],
)

history = model.fit(train_signals_data, train_labels, epochs=30)

Epoch 1/30


ValueError: in user code:

    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/astadnik/.virtualenvs/masters/lib/python3.10/sit

In [42]:
train_signals_data.shape, signals_data.shape

((1, 40, 28), (1, 1, 40, 23336))